### 模拟Federated SGD


In [1]:
import mxnet as mx
from mxnet import gluon
from mxnet.gluon import nn
from mxnet import autograd as ag
from mxnet import ndarray as nd
from Algorithm.CNN import CNN_Model
from Algorithm.MLP import MLP
from Tools import utils
import numpy as np

In [2]:
ctx = utils.try_all_gpus()
mlp = MLP()
mlp.load_parameters("mlp.params",ctx=ctx)
lenet = CNN_Model('LeNet')
lenet.load_parameters("LeNet.params",ctx=ctx)

def get_data(data_idx, batch_size):
    data = "E:\\PythonProjects\\Mxnet_FederatedLearning\\Fed_Client\\FedAvg_data\\train_data" + str(data_idx) + ".npy"
    label = "E:\\PythonProjects\\Mxnet_FederatedLearning\\Fed_Client\\FedAvg_data\\train_label" + str(data_idx) + ".npy"
    data = np.load(data)
    label = np.load(label)
    train_data = mx.io.NDArrayIter(data,label,batch_size=batch_size,shuffle=True)
    return train_data

def validate(net):
    mnist = mx.test_utils.get_mnist()
    val_data = mx.io.NDArrayIter(mnist['test_data'],mnist['test_label'],batch_size=100)
    metric = mx.metric.Accuracy()
    val_data.reset()
    for batch in val_data:
        data = gluon.utils.split_and_load(batch.data[0], ctx_list=ctx, batch_axis=0)
        label = gluon.utils.split_and_load(batch.label[0], ctx_list=ctx, batch_axis=0)
        outputs = []
        for x in data:
            outputs.append(net(x))
        metric.update(label, outputs)
    name,acc =metric.get()
    return acc

In [3]:
def train_till_acc(net, stop_acc, learning_rate, batch_size, epoch):
    round = 0
    for client in range(1000):
        client = client%100
        train_data = get_data(client, batch_size)
        softmax_cross_entropy_loss = gluon.loss.SoftmaxCrossEntropyLoss()
        trainer = trainer = gluon.Trainer(net.collect_params(), 'sgd', {'learning_rate': learning_rate})
        metric = mx.metric.Accuracy()
        stop_flag = False
        print("Round %d starting"%(round))
        for i in range(epoch):
            train_data.reset()
            for batch in train_data:
                data = gluon.utils.split_and_load(batch.data[0], ctx_list=ctx, batch_axis=0)
                label = gluon.utils.split_and_load(batch.label[0], ctx_list=ctx, batch_axis=0)
                outputs = []
                with ag.record():
                    for x, y in zip(data, label):
                        z = net(x)
                        loss = softmax_cross_entropy_loss(z, y)
                        loss.backward()
                        outputs.append(z)
                metric.update(label, outputs)
                trainer.step(batch.data[0].shape[0])
            name, acc = metric.get()
            metric.reset()
            print('training acc at epoch %d: %s=%f'%(i, name, acc))
        acc = validate(net)
        print("accuracy in test set %f"%(acc))
        if acc >= stop_acc:
            break
        round += 1
    print("round: ",round)

In [4]:
mlp.load_parameters("mlp.params",ctx=ctx)
train_till_acc(mlp,0.97,0.02,100,5)

t epoch 1: accuracy=0.988333
training acc at epoch 2: accuracy=0.990000
training acc at epoch 3: accuracy=0.993333
training acc at epoch 4: accuracy=0.993333
accuracy in test set 0.965700
Round 396 starting
training acc at epoch 0: accuracy=0.973333
training acc at epoch 1: accuracy=0.983333
training acc at epoch 2: accuracy=0.986667
training acc at epoch 3: accuracy=0.990000
training acc at epoch 4: accuracy=0.993333
accuracy in test set 0.965000
Round 397 starting
training acc at epoch 0: accuracy=0.991667
training acc at epoch 1: accuracy=0.993333
training acc at epoch 2: accuracy=0.998333
training acc at epoch 3: accuracy=0.998333
training acc at epoch 4: accuracy=0.998333
accuracy in test set 0.966400
Round 398 starting
training acc at epoch 0: accuracy=0.983333
training acc at epoch 1: accuracy=0.988333
training acc at epoch 2: accuracy=0.990000
training acc at epoch 3: accuracy=0.993333
training acc at epoch 4: accuracy=0.991667
accuracy in test set 0.964700
Round 399 starting
t

In [7]:
mlp.load_parameters("mlp.params",ctx=ctx)
train_till_acc(mlp,0.97,0.02,50,5)

t epoch 1: accuracy=0.968333
training acc at epoch 2: accuracy=0.975000
training acc at epoch 3: accuracy=0.978333
training acc at epoch 4: accuracy=0.980000
accuracy in test set 0.964600
Round 189 starting
training acc at epoch 0: accuracy=0.963333
training acc at epoch 1: accuracy=0.968333
training acc at epoch 2: accuracy=0.975000
training acc at epoch 3: accuracy=0.978333
training acc at epoch 4: accuracy=0.978333
accuracy in test set 0.959600
Round 190 starting
training acc at epoch 0: accuracy=0.971667
training acc at epoch 1: accuracy=0.980000
training acc at epoch 2: accuracy=0.983333
training acc at epoch 3: accuracy=0.985000
training acc at epoch 4: accuracy=0.988333
accuracy in test set 0.960800
Round 191 starting
training acc at epoch 0: accuracy=0.963333
training acc at epoch 1: accuracy=0.976667
training acc at epoch 2: accuracy=0.990000
training acc at epoch 3: accuracy=0.990000
training acc at epoch 4: accuracy=0.995000
accuracy in test set 0.961700
Round 192 starting
t

In [8]:
mlp.load_parameters("mlp.params",ctx=ctx)
train_till_acc(mlp,0.97,0.02,10,5)

g
training acc at epoch 0: accuracy=0.953333
training acc at epoch 1: accuracy=0.988333
training acc at epoch 2: accuracy=0.993333
training acc at epoch 3: accuracy=0.996667
training acc at epoch 4: accuracy=1.000000
accuracy in test set 0.951400
Round 41 starting
training acc at epoch 0: accuracy=0.946667
training acc at epoch 1: accuracy=0.988333
training acc at epoch 2: accuracy=0.993333
training acc at epoch 3: accuracy=0.998333
training acc at epoch 4: accuracy=1.000000
accuracy in test set 0.953700
Round 42 starting
training acc at epoch 0: accuracy=0.960000
training acc at epoch 1: accuracy=0.981667
training acc at epoch 2: accuracy=0.993333
training acc at epoch 3: accuracy=0.996667
training acc at epoch 4: accuracy=0.998333
accuracy in test set 0.949600
Round 43 starting
training acc at epoch 0: accuracy=0.948333
training acc at epoch 1: accuracy=0.975000
training acc at epoch 2: accuracy=0.993333
training acc at epoch 3: accuracy=0.998333
training acc at epoch 4: accuracy=1.0

In [9]:
mlp.load_parameters("mlp.params",ctx=ctx)
train_till_acc(mlp,0.97,0.02,200,5)

t epoch 1: accuracy=0.971667
training acc at epoch 2: accuracy=0.973333
training acc at epoch 3: accuracy=0.973333
training acc at epoch 4: accuracy=0.976667
accuracy in test set 0.967800
Round 785 starting
training acc at epoch 0: accuracy=0.975000
training acc at epoch 1: accuracy=0.978333
training acc at epoch 2: accuracy=0.980000
training acc at epoch 3: accuracy=0.980000
training acc at epoch 4: accuracy=0.983333
accuracy in test set 0.967500
Round 786 starting
training acc at epoch 0: accuracy=0.971667
training acc at epoch 1: accuracy=0.971667
training acc at epoch 2: accuracy=0.973333
training acc at epoch 3: accuracy=0.973333
training acc at epoch 4: accuracy=0.975000
accuracy in test set 0.968300
Round 787 starting
training acc at epoch 0: accuracy=0.981667
training acc at epoch 1: accuracy=0.986667
training acc at epoch 2: accuracy=0.986667
training acc at epoch 3: accuracy=0.990000
training acc at epoch 4: accuracy=0.991667
accuracy in test set 0.968900
Round 788 starting
t

In [10]:
mlp.load_parameters("mlp.params",ctx=ctx)
train_till_acc(mlp,0.97,0.01,200,5)

 epoch 1: accuracy=0.956667
training acc at epoch 2: accuracy=0.960000
training acc at epoch 3: accuracy=0.960000
training acc at epoch 4: accuracy=0.961667
accuracy in test set 0.957100
Round 925 starting
training acc at epoch 0: accuracy=0.963333
training acc at epoch 1: accuracy=0.963333
training acc at epoch 2: accuracy=0.963333
training acc at epoch 3: accuracy=0.965000
training acc at epoch 4: accuracy=0.966667
accuracy in test set 0.958200
Round 926 starting
training acc at epoch 0: accuracy=0.951667
training acc at epoch 1: accuracy=0.953333
training acc at epoch 2: accuracy=0.953333
training acc at epoch 3: accuracy=0.956667
training acc at epoch 4: accuracy=0.958333
accuracy in test set 0.958400
Round 927 starting
training acc at epoch 0: accuracy=0.961667
training acc at epoch 1: accuracy=0.965000
training acc at epoch 2: accuracy=0.965000
training acc at epoch 3: accuracy=0.965000
training acc at epoch 4: accuracy=0.965000
accuracy in test set 0.958500
Round 928 starting
tr

In [11]:
train_till_acc(mlp,0.97,0.01,200,5)

t epoch 1: accuracy=0.980000
training acc at epoch 2: accuracy=0.980000
training acc at epoch 3: accuracy=0.980000
training acc at epoch 4: accuracy=0.980000
accuracy in test set 0.968800
Round 591 starting
training acc at epoch 0: accuracy=0.981667
training acc at epoch 1: accuracy=0.983333
training acc at epoch 2: accuracy=0.983333
training acc at epoch 3: accuracy=0.981667
training acc at epoch 4: accuracy=0.983333
accuracy in test set 0.968000
Round 592 starting
training acc at epoch 0: accuracy=0.968333
training acc at epoch 1: accuracy=0.970000
training acc at epoch 2: accuracy=0.971667
training acc at epoch 3: accuracy=0.973333
training acc at epoch 4: accuracy=0.976667
accuracy in test set 0.967900
Round 593 starting
training acc at epoch 0: accuracy=0.975000
training acc at epoch 1: accuracy=0.976667
training acc at epoch 2: accuracy=0.976667
training acc at epoch 3: accuracy=0.978333
training acc at epoch 4: accuracy=0.981667
accuracy in test set 0.968100
Round 594 starting
t

In [12]:
mlp.load_parameters("mlp.params",ctx=ctx)
train_till_acc(mlp,0.97,0.01,100,5)

t epoch 1: accuracy=0.971667
training acc at epoch 2: accuracy=0.973333
training acc at epoch 3: accuracy=0.975000
training acc at epoch 4: accuracy=0.978333
accuracy in test set 0.967600
Round 785 starting
training acc at epoch 0: accuracy=0.978333
training acc at epoch 1: accuracy=0.978333
training acc at epoch 2: accuracy=0.980000
training acc at epoch 3: accuracy=0.981667
training acc at epoch 4: accuracy=0.980000
accuracy in test set 0.967800
Round 786 starting
training acc at epoch 0: accuracy=0.971667
training acc at epoch 1: accuracy=0.971667
training acc at epoch 2: accuracy=0.973333
training acc at epoch 3: accuracy=0.975000
training acc at epoch 4: accuracy=0.975000
accuracy in test set 0.968300
Round 787 starting
training acc at epoch 0: accuracy=0.983333
training acc at epoch 1: accuracy=0.986667
training acc at epoch 2: accuracy=0.990000
training acc at epoch 3: accuracy=0.990000
training acc at epoch 4: accuracy=0.991667
accuracy in test set 0.968800
Round 788 starting
t

In [13]:
mlp.load_parameters("mlp.params",ctx=ctx)
train_till_acc(mlp,0.97,0.01,50 ,5)

t epoch 1: accuracy=0.986667
training acc at epoch 2: accuracy=0.991667
training acc at epoch 3: accuracy=0.991667
training acc at epoch 4: accuracy=0.993333
accuracy in test set 0.965700
Round 396 starting
training acc at epoch 0: accuracy=0.973333
training acc at epoch 1: accuracy=0.985000
training acc at epoch 2: accuracy=0.985000
training acc at epoch 3: accuracy=0.990000
training acc at epoch 4: accuracy=0.993333
accuracy in test set 0.965200
Round 397 starting
training acc at epoch 0: accuracy=0.991667
training acc at epoch 1: accuracy=0.995000
training acc at epoch 2: accuracy=0.998333
training acc at epoch 3: accuracy=0.998333
training acc at epoch 4: accuracy=0.998333
accuracy in test set 0.966100
Round 398 starting
training acc at epoch 0: accuracy=0.985000
training acc at epoch 1: accuracy=0.988333
training acc at epoch 2: accuracy=0.991667
training acc at epoch 3: accuracy=0.991667
training acc at epoch 4: accuracy=0.991667
accuracy in test set 0.964700
Round 399 starting
t

In [14]:
mlp.load_parameters("mlp.params",ctx=ctx)
train_till_acc(mlp,0.97,0.01,10,5)

=0.941667
training acc at epoch 1: accuracy=0.975000
training acc at epoch 2: accuracy=0.985000
training acc at epoch 3: accuracy=0.995000
training acc at epoch 4: accuracy=0.996667
accuracy in test set 0.957200
Round 76 starting
training acc at epoch 0: accuracy=0.945000
training acc at epoch 1: accuracy=0.965000
training acc at epoch 2: accuracy=0.970000
training acc at epoch 3: accuracy=0.978333
training acc at epoch 4: accuracy=0.986667
accuracy in test set 0.954400
Round 77 starting
training acc at epoch 0: accuracy=0.943333
training acc at epoch 1: accuracy=0.953333
training acc at epoch 2: accuracy=0.975000
training acc at epoch 3: accuracy=0.980000
training acc at epoch 4: accuracy=0.990000
accuracy in test set 0.954200
Round 78 starting
training acc at epoch 0: accuracy=0.941667
training acc at epoch 1: accuracy=0.961667
training acc at epoch 2: accuracy=0.980000
training acc at epoch 3: accuracy=0.986667
training acc at epoch 4: accuracy=0.993333
accuracy in test set 0.950100

In [15]:
mlp.load_parameters("mlp.params",ctx=ctx)
train_till_acc(mlp,0.97,0.04,200 ,5)

t epoch 1: accuracy=0.983333
training acc at epoch 2: accuracy=0.988333
training acc at epoch 3: accuracy=0.993333
training acc at epoch 4: accuracy=0.993333
accuracy in test set 0.965500
Round 396 starting
training acc at epoch 0: accuracy=0.975000
training acc at epoch 1: accuracy=0.985000
training acc at epoch 2: accuracy=0.988333
training acc at epoch 3: accuracy=0.988333
training acc at epoch 4: accuracy=0.993333
accuracy in test set 0.965100
Round 397 starting
training acc at epoch 0: accuracy=0.991667
training acc at epoch 1: accuracy=0.991667
training acc at epoch 2: accuracy=0.998333
training acc at epoch 3: accuracy=0.998333
training acc at epoch 4: accuracy=0.998333
accuracy in test set 0.966800
Round 398 starting
training acc at epoch 0: accuracy=0.981667
training acc at epoch 1: accuracy=0.986667
training acc at epoch 2: accuracy=0.991667
training acc at epoch 3: accuracy=0.991667
training acc at epoch 4: accuracy=0.991667
accuracy in test set 0.964400
Round 399 starting
t

In [16]:
mlp.load_parameters("mlp.params",ctx=ctx)
train_till_acc(mlp,0.97,0.04,100,5)

t epoch 1: accuracy=0.981667
training acc at epoch 2: accuracy=0.981667
training acc at epoch 3: accuracy=0.988333
training acc at epoch 4: accuracy=0.988333
accuracy in test set 0.963700
Round 213 starting
training acc at epoch 0: accuracy=0.958333
training acc at epoch 1: accuracy=0.970000
training acc at epoch 2: accuracy=0.980000
training acc at epoch 3: accuracy=0.985000
training acc at epoch 4: accuracy=0.985000
accuracy in test set 0.966400
Round 214 starting
training acc at epoch 0: accuracy=0.953333
training acc at epoch 1: accuracy=0.961667
training acc at epoch 2: accuracy=0.976667
training acc at epoch 3: accuracy=0.981667
training acc at epoch 4: accuracy=0.988333
accuracy in test set 0.963500
Round 215 starting
training acc at epoch 0: accuracy=0.966667
training acc at epoch 1: accuracy=0.975000
training acc at epoch 2: accuracy=0.981667
training acc at epoch 3: accuracy=0.985000
training acc at epoch 4: accuracy=0.990000
accuracy in test set 0.965100
Round 216 starting
t

In [18]:
mlp.load_parameters("mlp.params",ctx=ctx)
train_till_acc(mlp,0.97,0.04,50,5)

t epoch 1: accuracy=0.980000
training acc at epoch 2: accuracy=0.991667
training acc at epoch 3: accuracy=0.996667
training acc at epoch 4: accuracy=1.000000
accuracy in test set 0.965100
Round 129 starting
training acc at epoch 0: accuracy=0.958333
training acc at epoch 1: accuracy=0.971667
training acc at epoch 2: accuracy=0.980000
training acc at epoch 3: accuracy=0.990000
training acc at epoch 4: accuracy=0.993333
accuracy in test set 0.965600
Round 130 starting
training acc at epoch 0: accuracy=0.963333
training acc at epoch 1: accuracy=0.985000
training acc at epoch 2: accuracy=0.990000
training acc at epoch 3: accuracy=0.996667
training acc at epoch 4: accuracy=1.000000
accuracy in test set 0.963900
Round 131 starting
training acc at epoch 0: accuracy=0.975000
training acc at epoch 1: accuracy=0.983333
training acc at epoch 2: accuracy=0.993333
training acc at epoch 3: accuracy=0.996667
training acc at epoch 4: accuracy=0.998333
accuracy in test set 0.966000
Round 132 starting
t

In [19]:
mlp.load_parameters("mlp.params",ctx=ctx)
train_till_acc(mlp,0.97,0.04,10,5)

1 starting
training acc at epoch 0: accuracy=0.961667
training acc at epoch 1: accuracy=0.985000
training acc at epoch 2: accuracy=0.995000
training acc at epoch 3: accuracy=1.000000
training acc at epoch 4: accuracy=1.000000
accuracy in test set 0.955100
Round 32 starting
training acc at epoch 0: accuracy=0.951667
training acc at epoch 1: accuracy=0.986667
training acc at epoch 2: accuracy=0.998333
training acc at epoch 3: accuracy=1.000000
training acc at epoch 4: accuracy=1.000000
accuracy in test set 0.950400
Round 33 starting
training acc at epoch 0: accuracy=0.943333
training acc at epoch 1: accuracy=0.985000
training acc at epoch 2: accuracy=0.995000
training acc at epoch 3: accuracy=0.998333
training acc at epoch 4: accuracy=1.000000
accuracy in test set 0.954000
Round 34 starting
training acc at epoch 0: accuracy=0.941667
training acc at epoch 1: accuracy=0.983333
training acc at epoch 2: accuracy=0.995000
training acc at epoch 3: accuracy=0.996667
training acc at epoch 4: acc

In [20]:
lenet.load_parameters("LeNet.params",ctx=ctx)
train_till_acc(lenet, 0.988, 0.01, 200, 5)

 epoch 1: accuracy=0.990000
training acc at epoch 2: accuracy=0.988333
training acc at epoch 3: accuracy=0.991667
training acc at epoch 4: accuracy=0.995000
accuracy in test set 0.979600
Round 925 starting
training acc at epoch 0: accuracy=0.985000
training acc at epoch 1: accuracy=0.991667
training acc at epoch 2: accuracy=0.995000
training acc at epoch 3: accuracy=0.998333
training acc at epoch 4: accuracy=1.000000
accuracy in test set 0.982600
Round 926 starting
training acc at epoch 0: accuracy=0.981667
training acc at epoch 1: accuracy=0.990000
training acc at epoch 2: accuracy=0.991667
training acc at epoch 3: accuracy=0.990000
training acc at epoch 4: accuracy=0.993333
accuracy in test set 0.980200
Round 927 starting
training acc at epoch 0: accuracy=0.985000
training acc at epoch 1: accuracy=0.990000
training acc at epoch 2: accuracy=0.991667
training acc at epoch 3: accuracy=0.993333
training acc at epoch 4: accuracy=0.996667
accuracy in test set 0.980000
Round 928 starting
tr

In [21]:
train_till_acc(lenet, 0.988, 0.01, 200, 5)

 epoch 1: accuracy=0.993333
training acc at epoch 2: accuracy=0.993333
training acc at epoch 3: accuracy=0.996667
training acc at epoch 4: accuracy=0.996667
accuracy in test set 0.986200
Round 925 starting
training acc at epoch 0: accuracy=0.996667
training acc at epoch 1: accuracy=1.000000
training acc at epoch 2: accuracy=1.000000
training acc at epoch 3: accuracy=1.000000
training acc at epoch 4: accuracy=1.000000
accuracy in test set 0.985400
Round 926 starting
training acc at epoch 0: accuracy=0.988333
training acc at epoch 1: accuracy=0.995000
training acc at epoch 2: accuracy=0.998333
training acc at epoch 3: accuracy=0.998333
training acc at epoch 4: accuracy=0.998333
accuracy in test set 0.986200
Round 927 starting
training acc at epoch 0: accuracy=0.991667
training acc at epoch 1: accuracy=0.995000
training acc at epoch 2: accuracy=0.996667
training acc at epoch 3: accuracy=0.998333
training acc at epoch 4: accuracy=0.996667
accuracy in test set 0.984900
Round 928 starting
tr

In [22]:
lenet.load_parameters("LeNet.params",ctx=ctx)
train_till_acc(lenet, 0.988, 0.01, 100, 5)

 epoch 1: accuracy=0.998333
training acc at epoch 2: accuracy=0.996667
training acc at epoch 3: accuracy=1.000000
training acc at epoch 4: accuracy=1.000000
accuracy in test set 0.986300
Round 925 starting
training acc at epoch 0: accuracy=0.991667
training acc at epoch 1: accuracy=0.996667
training acc at epoch 2: accuracy=1.000000
training acc at epoch 3: accuracy=1.000000
training acc at epoch 4: accuracy=1.000000
accuracy in test set 0.985700
Round 926 starting
training acc at epoch 0: accuracy=0.990000
training acc at epoch 1: accuracy=0.995000
training acc at epoch 2: accuracy=0.998333
training acc at epoch 3: accuracy=1.000000
training acc at epoch 4: accuracy=1.000000
accuracy in test set 0.984800
Round 927 starting
training acc at epoch 0: accuracy=0.986667
training acc at epoch 1: accuracy=0.995000
training acc at epoch 2: accuracy=0.996667
training acc at epoch 3: accuracy=0.996667
training acc at epoch 4: accuracy=0.998333
accuracy in test set 0.984300
Round 928 starting
tr

In [23]:
train_till_acc(lenet, 0.988, 0.01, 100, 5)

 epoch 0: accuracy=0.996667
training acc at epoch 1: accuracy=1.000000
training acc at epoch 2: accuracy=1.000000
training acc at epoch 3: accuracy=1.000000
training acc at epoch 4: accuracy=1.000000
accuracy in test set 0.986600
Round 132 starting
training acc at epoch 0: accuracy=0.990000
training acc at epoch 1: accuracy=0.998333
training acc at epoch 2: accuracy=1.000000
training acc at epoch 3: accuracy=1.000000
training acc at epoch 4: accuracy=1.000000
accuracy in test set 0.985000
Round 133 starting
training acc at epoch 0: accuracy=0.990000
training acc at epoch 1: accuracy=0.998333
training acc at epoch 2: accuracy=1.000000
training acc at epoch 3: accuracy=1.000000
training acc at epoch 4: accuracy=1.000000
accuracy in test set 0.985100
Round 134 starting
training acc at epoch 0: accuracy=0.993333
training acc at epoch 1: accuracy=0.998333
training acc at epoch 2: accuracy=1.000000
training acc at epoch 3: accuracy=1.000000
training acc at epoch 4: accuracy=1.000000
accuracy

KeyboardInterrupt: 

In [24]:
lenet.load_parameters("LeNet.params",ctx=ctx)
train_till_acc(lenet, 0.988, 0.02, 10, 5)

t epoch 1: accuracy=0.998333
training acc at epoch 2: accuracy=0.998333
training acc at epoch 3: accuracy=1.000000
training acc at epoch 4: accuracy=1.000000
accuracy in test set 0.985400
Round 160 starting
training acc at epoch 0: accuracy=0.981667
training acc at epoch 1: accuracy=0.998333
training acc at epoch 2: accuracy=0.996667
training acc at epoch 3: accuracy=1.000000
training acc at epoch 4: accuracy=1.000000
accuracy in test set 0.984200
Round 161 starting
training acc at epoch 0: accuracy=0.973333
training acc at epoch 1: accuracy=0.995000
training acc at epoch 2: accuracy=0.998333
training acc at epoch 3: accuracy=1.000000
training acc at epoch 4: accuracy=1.000000
accuracy in test set 0.984800
Round 162 starting
training acc at epoch 0: accuracy=0.988333
training acc at epoch 1: accuracy=1.000000
training acc at epoch 2: accuracy=1.000000
training acc at epoch 3: accuracy=1.000000
training acc at epoch 4: accuracy=1.000000
accuracy in test set 0.985000
Round 163 starting
t

In [25]:
lenet.load_parameters("LeNet.params",ctx=ctx)
train_till_acc(lenet, 0.988, 0.02, 50, 5)

t epoch 1: accuracy=0.995000
training acc at epoch 2: accuracy=0.998333
training acc at epoch 3: accuracy=1.000000
training acc at epoch 4: accuracy=1.000000
accuracy in test set 0.985500
Round 458 starting
training acc at epoch 0: accuracy=0.983333
training acc at epoch 1: accuracy=0.995000
training acc at epoch 2: accuracy=0.998333
training acc at epoch 3: accuracy=1.000000
training acc at epoch 4: accuracy=1.000000
accuracy in test set 0.984200
Round 459 starting
training acc at epoch 0: accuracy=0.983333
training acc at epoch 1: accuracy=0.998333
training acc at epoch 2: accuracy=0.998333
training acc at epoch 3: accuracy=0.998333
training acc at epoch 4: accuracy=1.000000
accuracy in test set 0.984200
Round 460 starting
training acc at epoch 0: accuracy=0.983333
training acc at epoch 1: accuracy=0.998333
training acc at epoch 2: accuracy=1.000000
training acc at epoch 3: accuracy=1.000000
training acc at epoch 4: accuracy=1.000000
accuracy in test set 0.983100
Round 461 starting
t

In [26]:
lenet.load_parameters("LeNet.params",ctx=ctx)
train_till_acc(lenet, 0.988, 0.02, 100, 5)

t epoch 1: accuracy=0.996667
training acc at epoch 2: accuracy=1.000000
training acc at epoch 3: accuracy=1.000000
training acc at epoch 4: accuracy=1.000000
accuracy in test set 0.986700
Round 635 starting
training acc at epoch 0: accuracy=0.993333
training acc at epoch 1: accuracy=0.993333
training acc at epoch 2: accuracy=1.000000
training acc at epoch 3: accuracy=1.000000
training acc at epoch 4: accuracy=1.000000
accuracy in test set 0.987000
Round 636 starting
training acc at epoch 0: accuracy=0.995000
training acc at epoch 1: accuracy=1.000000
training acc at epoch 2: accuracy=1.000000
training acc at epoch 3: accuracy=1.000000
training acc at epoch 4: accuracy=1.000000
accuracy in test set 0.985500
Round 637 starting
training acc at epoch 0: accuracy=0.985000
training acc at epoch 1: accuracy=0.995000
training acc at epoch 2: accuracy=1.000000
training acc at epoch 3: accuracy=1.000000
training acc at epoch 4: accuracy=1.000000
accuracy in test set 0.985500
Round 638 starting
t

In [27]:
lenet.load_parameters("LeNet.params",ctx=ctx)
train_till_acc(lenet, 0.988, 0.02, 200, 5)

 epoch 1: accuracy=0.996667
training acc at epoch 2: accuracy=1.000000
training acc at epoch 3: accuracy=1.000000
training acc at epoch 4: accuracy=1.000000
accuracy in test set 0.986500
Round 925 starting
training acc at epoch 0: accuracy=0.991667
training acc at epoch 1: accuracy=0.998333
training acc at epoch 2: accuracy=1.000000
training acc at epoch 3: accuracy=1.000000
training acc at epoch 4: accuracy=1.000000
accuracy in test set 0.986000
Round 926 starting
training acc at epoch 0: accuracy=0.983333
training acc at epoch 1: accuracy=0.993333
training acc at epoch 2: accuracy=0.996667
training acc at epoch 3: accuracy=1.000000
training acc at epoch 4: accuracy=1.000000
accuracy in test set 0.985100
Round 927 starting
training acc at epoch 0: accuracy=0.985000
training acc at epoch 1: accuracy=0.995000
training acc at epoch 2: accuracy=0.996667
training acc at epoch 3: accuracy=0.998333
training acc at epoch 4: accuracy=0.996667
accuracy in test set 0.983200
Round 928 starting
tr

In [28]:
train_till_acc(lenet, 0.988, 0.02, 200, 5)

t epoch 1: accuracy=1.000000
training acc at epoch 2: accuracy=1.000000
training acc at epoch 3: accuracy=1.000000
training acc at epoch 4: accuracy=1.000000
accuracy in test set 0.986600
Round 355 starting
training acc at epoch 0: accuracy=0.995000
training acc at epoch 1: accuracy=0.998333
training acc at epoch 2: accuracy=1.000000
training acc at epoch 3: accuracy=1.000000
training acc at epoch 4: accuracy=1.000000
accuracy in test set 0.985900
Round 356 starting
training acc at epoch 0: accuracy=0.993333
training acc at epoch 1: accuracy=1.000000
training acc at epoch 2: accuracy=1.000000
training acc at epoch 3: accuracy=1.000000
training acc at epoch 4: accuracy=1.000000
accuracy in test set 0.985600
Round 357 starting
training acc at epoch 0: accuracy=0.986667
training acc at epoch 1: accuracy=0.995000
training acc at epoch 2: accuracy=1.000000
training acc at epoch 3: accuracy=1.000000
training acc at epoch 4: accuracy=1.000000
accuracy in test set 0.986000
Round 358 starting
t

In [29]:
lenet.load_parameters("LeNet.params",ctx=ctx)
train_till_acc(lenet, 0.988, 0.04, 200, 5)

t epoch 1: accuracy=1.000000
training acc at epoch 2: accuracy=1.000000
training acc at epoch 3: accuracy=1.000000
training acc at epoch 4: accuracy=1.000000
accuracy in test set 0.987600
Round 835 starting
training acc at epoch 0: accuracy=0.995000
training acc at epoch 1: accuracy=1.000000
training acc at epoch 2: accuracy=1.000000
training acc at epoch 3: accuracy=1.000000
training acc at epoch 4: accuracy=1.000000
accuracy in test set 0.987100
Round 836 starting
training acc at epoch 0: accuracy=0.995000
training acc at epoch 1: accuracy=1.000000
training acc at epoch 2: accuracy=1.000000
training acc at epoch 3: accuracy=1.000000
training acc at epoch 4: accuracy=1.000000
accuracy in test set 0.985800
Round 837 starting
training acc at epoch 0: accuracy=0.993333
training acc at epoch 1: accuracy=0.996667
training acc at epoch 2: accuracy=1.000000
training acc at epoch 3: accuracy=1.000000
training acc at epoch 4: accuracy=1.000000
accuracy in test set 0.986800
Round 838 starting
t

In [31]:
lenet.load_parameters("LeNet.params",ctx=ctx)
train_till_acc(lenet, 0.988, 0.04, 100, 5)

t epoch 1: accuracy=1.000000
training acc at epoch 2: accuracy=1.000000
training acc at epoch 3: accuracy=1.000000
training acc at epoch 4: accuracy=1.000000
accuracy in test set 0.987500
Round 649 starting
training acc at epoch 0: accuracy=0.995000
training acc at epoch 1: accuracy=1.000000
training acc at epoch 2: accuracy=1.000000
training acc at epoch 3: accuracy=1.000000
training acc at epoch 4: accuracy=1.000000
accuracy in test set 0.985800
Round 650 starting
training acc at epoch 0: accuracy=0.996667
training acc at epoch 1: accuracy=1.000000
training acc at epoch 2: accuracy=1.000000
training acc at epoch 3: accuracy=1.000000
training acc at epoch 4: accuracy=1.000000
accuracy in test set 0.987000
Round 651 starting
training acc at epoch 0: accuracy=0.990000
training acc at epoch 1: accuracy=1.000000
training acc at epoch 2: accuracy=1.000000
training acc at epoch 3: accuracy=1.000000
training acc at epoch 4: accuracy=1.000000
accuracy in test set 0.987400
Round 652 starting
t

In [32]:
lenet.load_parameters("LeNet.params",ctx=ctx)
train_till_acc(lenet, 0.988, 0.04, 50, 5)

t epoch 1: accuracy=0.993333
training acc at epoch 2: accuracy=1.000000
training acc at epoch 3: accuracy=1.000000
training acc at epoch 4: accuracy=1.000000
accuracy in test set 0.983700
Round 174 starting
training acc at epoch 0: accuracy=0.983333
training acc at epoch 1: accuracy=0.995000
training acc at epoch 2: accuracy=0.998333
training acc at epoch 3: accuracy=1.000000
training acc at epoch 4: accuracy=1.000000
accuracy in test set 0.985200
Round 175 starting
training acc at epoch 0: accuracy=0.985000
training acc at epoch 1: accuracy=0.996667
training acc at epoch 2: accuracy=0.996667
training acc at epoch 3: accuracy=0.998333
training acc at epoch 4: accuracy=0.998333
accuracy in test set 0.984600
Round 176 starting
training acc at epoch 0: accuracy=0.980000
training acc at epoch 1: accuracy=0.998333
training acc at epoch 2: accuracy=0.996667
training acc at epoch 3: accuracy=1.000000
training acc at epoch 4: accuracy=1.000000
accuracy in test set 0.984800
Round 177 starting
t

In [33]:
lenet.load_parameters("LeNet.params",ctx=ctx)
train_till_acc(lenet, 0.988, 0.04, 10, 5)

t epoch 1: accuracy=1.000000
training acc at epoch 2: accuracy=1.000000
training acc at epoch 3: accuracy=1.000000
training acc at epoch 4: accuracy=1.000000
accuracy in test set 0.983800
Round 161 starting
training acc at epoch 0: accuracy=0.986667
training acc at epoch 1: accuracy=0.996667
training acc at epoch 2: accuracy=1.000000
training acc at epoch 3: accuracy=1.000000
training acc at epoch 4: accuracy=1.000000
accuracy in test set 0.985800
Round 162 starting
training acc at epoch 0: accuracy=0.983333
training acc at epoch 1: accuracy=0.995000
training acc at epoch 2: accuracy=0.998333
training acc at epoch 3: accuracy=1.000000
training acc at epoch 4: accuracy=1.000000
accuracy in test set 0.986500
Round 163 starting
training acc at epoch 0: accuracy=0.985000
training acc at epoch 1: accuracy=0.998333
training acc at epoch 2: accuracy=1.000000
training acc at epoch 3: accuracy=1.000000
training acc at epoch 4: accuracy=1.000000
accuracy in test set 0.985300
Round 164 starting
t

In [34]:
lenet.load_parameters("LeNet.params",ctx=ctx)
train_till_acc(lenet, 0.988, 0.01, 10, 5)

t epoch 1: accuracy=1.000000
training acc at epoch 2: accuracy=1.000000
training acc at epoch 3: accuracy=1.000000
training acc at epoch 4: accuracy=1.000000
accuracy in test set 0.983500
Round 196 starting
training acc at epoch 0: accuracy=0.988333
training acc at epoch 1: accuracy=0.995000
training acc at epoch 2: accuracy=0.998333
training acc at epoch 3: accuracy=1.000000
training acc at epoch 4: accuracy=1.000000
accuracy in test set 0.983900
Round 197 starting
training acc at epoch 0: accuracy=0.993333
training acc at epoch 1: accuracy=1.000000
training acc at epoch 2: accuracy=1.000000
training acc at epoch 3: accuracy=1.000000
training acc at epoch 4: accuracy=1.000000
accuracy in test set 0.980000
Round 198 starting
training acc at epoch 0: accuracy=0.996667
training acc at epoch 1: accuracy=1.000000
training acc at epoch 2: accuracy=1.000000
training acc at epoch 3: accuracy=1.000000
training acc at epoch 4: accuracy=1.000000
accuracy in test set 0.983800
Round 199 starting
t

In [35]:
lenet.load_parameters("LeNet.params",ctx=ctx)
train_till_acc(lenet, 0.988, 0.01, 50, 5)

t epoch 1: accuracy=0.998333
training acc at epoch 2: accuracy=1.000000
training acc at epoch 3: accuracy=1.000000
training acc at epoch 4: accuracy=1.000000
accuracy in test set 0.987500
Round 655 starting
training acc at epoch 0: accuracy=0.991667
training acc at epoch 1: accuracy=1.000000
training acc at epoch 2: accuracy=1.000000
training acc at epoch 3: accuracy=1.000000
training acc at epoch 4: accuracy=1.000000
accuracy in test set 0.985400
Round 656 starting
training acc at epoch 0: accuracy=0.993333
training acc at epoch 1: accuracy=1.000000
training acc at epoch 2: accuracy=1.000000
training acc at epoch 3: accuracy=1.000000
training acc at epoch 4: accuracy=1.000000
accuracy in test set 0.984700
Round 657 starting
training acc at epoch 0: accuracy=0.985000
training acc at epoch 1: accuracy=0.993333
training acc at epoch 2: accuracy=0.995000
training acc at epoch 3: accuracy=1.000000
training acc at epoch 4: accuracy=1.000000
accuracy in test set 0.982700
Round 658 starting
t

In [36]:
lenet.load_parameters("LeNet.params",ctx=ctx)
train_till_acc(lenet, 0.988, 0.01, 100, 5)

 epoch 1: accuracy=0.995000
training acc at epoch 2: accuracy=0.998333
training acc at epoch 3: accuracy=1.000000
training acc at epoch 4: accuracy=1.000000
accuracy in test set 0.985700
Round 925 starting
training acc at epoch 0: accuracy=0.993333
training acc at epoch 1: accuracy=1.000000
training acc at epoch 2: accuracy=1.000000
training acc at epoch 3: accuracy=1.000000
training acc at epoch 4: accuracy=1.000000
accuracy in test set 0.984700
Round 926 starting
training acc at epoch 0: accuracy=0.985000
training acc at epoch 1: accuracy=0.995000
training acc at epoch 2: accuracy=0.996667
training acc at epoch 3: accuracy=1.000000
training acc at epoch 4: accuracy=1.000000
accuracy in test set 0.984600
Round 927 starting
training acc at epoch 0: accuracy=0.991667
training acc at epoch 1: accuracy=0.996667
training acc at epoch 2: accuracy=0.996667
training acc at epoch 3: accuracy=0.998333
training acc at epoch 4: accuracy=0.998333
accuracy in test set 0.983800
Round 928 starting
tr

In [37]:
train_till_acc(lenet, 0.988, 0.01, 100, 5)

t epoch 1: accuracy=1.000000
training acc at epoch 2: accuracy=1.000000
training acc at epoch 3: accuracy=1.000000
training acc at epoch 4: accuracy=1.000000
accuracy in test set 0.987100
Round 755 starting
training acc at epoch 0: accuracy=0.996667
training acc at epoch 1: accuracy=0.998333
training acc at epoch 2: accuracy=1.000000
training acc at epoch 3: accuracy=1.000000
training acc at epoch 4: accuracy=1.000000
accuracy in test set 0.986300
Round 756 starting
training acc at epoch 0: accuracy=0.996667
training acc at epoch 1: accuracy=1.000000
training acc at epoch 2: accuracy=1.000000
training acc at epoch 3: accuracy=1.000000
training acc at epoch 4: accuracy=1.000000
accuracy in test set 0.986500
Round 757 starting
training acc at epoch 0: accuracy=0.988333
training acc at epoch 1: accuracy=0.996667
training acc at epoch 2: accuracy=1.000000
training acc at epoch 3: accuracy=1.000000
training acc at epoch 4: accuracy=1.000000
accuracy in test set 0.985900
Round 758 starting
t

In [38]:
lenet.load_parameters("LeNet.params",ctx=ctx)
train_till_acc(lenet, 0.988, 0.01, 200, 5)

 epoch 1: accuracy=0.988333
training acc at epoch 2: accuracy=0.988333
training acc at epoch 3: accuracy=0.991667
training acc at epoch 4: accuracy=0.993333
accuracy in test set 0.982900
Round 925 starting
training acc at epoch 0: accuracy=0.986667
training acc at epoch 1: accuracy=0.993333
training acc at epoch 2: accuracy=0.998333
training acc at epoch 3: accuracy=0.998333
training acc at epoch 4: accuracy=1.000000
accuracy in test set 0.982600
Round 926 starting
training acc at epoch 0: accuracy=0.983333
training acc at epoch 1: accuracy=0.990000
training acc at epoch 2: accuracy=0.988333
training acc at epoch 3: accuracy=0.993333
training acc at epoch 4: accuracy=0.991667
accuracy in test set 0.980600
Round 927 starting
training acc at epoch 0: accuracy=0.986667
training acc at epoch 1: accuracy=0.991667
training acc at epoch 2: accuracy=0.991667
training acc at epoch 3: accuracy=0.996667
training acc at epoch 4: accuracy=0.995000
accuracy in test set 0.979300
Round 928 starting
tr

In [39]:
lenet.load_parameters("LeNet.params",ctx=ctx)
train_till_acc(lenet, 0.988, 0.01, 200, 5)

 epoch 1: accuracy=0.988333
training acc at epoch 2: accuracy=0.986667
training acc at epoch 3: accuracy=0.991667
training acc at epoch 4: accuracy=0.988333
accuracy in test set 0.981400
Round 925 starting
training acc at epoch 0: accuracy=0.985000
training acc at epoch 1: accuracy=0.993333
training acc at epoch 2: accuracy=0.996667
training acc at epoch 3: accuracy=0.996667
training acc at epoch 4: accuracy=0.998333
accuracy in test set 0.981400
Round 926 starting
training acc at epoch 0: accuracy=0.976667
training acc at epoch 1: accuracy=0.988333
training acc at epoch 2: accuracy=0.990000
training acc at epoch 3: accuracy=0.993333
training acc at epoch 4: accuracy=0.993333
accuracy in test set 0.981500
Round 927 starting
training acc at epoch 0: accuracy=0.985000
training acc at epoch 1: accuracy=0.988333
training acc at epoch 2: accuracy=0.990000
training acc at epoch 3: accuracy=0.991667
training acc at epoch 4: accuracy=0.993333
accuracy in test set 0.979200
Round 928 starting
tr

In [40]:
train_till_acc(lenet, 0.988, 0.01, 200, 5)

 epoch 1: accuracy=0.990000
training acc at epoch 2: accuracy=0.998333
training acc at epoch 3: accuracy=0.993333
training acc at epoch 4: accuracy=1.000000
accuracy in test set 0.986500
Round 925 starting
training acc at epoch 0: accuracy=0.996667
training acc at epoch 1: accuracy=1.000000
training acc at epoch 2: accuracy=1.000000
training acc at epoch 3: accuracy=1.000000
training acc at epoch 4: accuracy=1.000000
accuracy in test set 0.986700
Round 926 starting
training acc at epoch 0: accuracy=0.990000
training acc at epoch 1: accuracy=0.995000
training acc at epoch 2: accuracy=0.998333
training acc at epoch 3: accuracy=0.998333
training acc at epoch 4: accuracy=0.998333
accuracy in test set 0.984400
Round 927 starting
training acc at epoch 0: accuracy=0.991667
training acc at epoch 1: accuracy=0.996667
training acc at epoch 2: accuracy=0.996667
training acc at epoch 3: accuracy=0.998333
training acc at epoch 4: accuracy=0.996667
accuracy in test set 0.984000
Round 928 starting
tr

In [41]:
train_till_acc(lenet, 0.988, 0.01, 200, 5)

t epoch 1: accuracy=0.998333
training acc at epoch 2: accuracy=1.000000
training acc at epoch 3: accuracy=1.000000
training acc at epoch 4: accuracy=1.000000
accuracy in test set 0.984800
Round 480 starting
training acc at epoch 0: accuracy=0.996667
training acc at epoch 1: accuracy=1.000000
training acc at epoch 2: accuracy=1.000000
training acc at epoch 3: accuracy=1.000000
training acc at epoch 4: accuracy=1.000000
accuracy in test set 0.984900
Round 481 starting
training acc at epoch 0: accuracy=0.988333
training acc at epoch 1: accuracy=0.988333
training acc at epoch 2: accuracy=0.995000
training acc at epoch 3: accuracy=0.998333
training acc at epoch 4: accuracy=0.996667
accuracy in test set 0.984100
Round 482 starting
training acc at epoch 0: accuracy=0.993333
training acc at epoch 1: accuracy=0.996667
training acc at epoch 2: accuracy=0.998333
training acc at epoch 3: accuracy=0.998333
training acc at epoch 4: accuracy=0.998333
accuracy in test set 0.984500
Round 483 starting
t